<a href="https://colab.research.google.com/github/Enchiii/ImageClassification/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

In [ ]:
IMG_SIZE = (224, 224)

In [ ]:
train_data_path = 'data/seg_train/seg_train'
test_data_path = 'data/seg_test/seg_test'
pred_data_path = 'data/seg_pred'

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    zoom_range=0.2,
    rotation_range=0.4,
    validation_split=0.8
)

In [ ]:
train_data = datagen.flow_from_directory(
    train_data_path,
    target_size=IMG_SIZE,
    subset='training'
)
test_data = datagen.flow_from_directory(
    test_data_path,
    target_size=IMG_SIZE,
    subset='training'
)
pred_data = datagen.flow_from_directory(
    pred_data_path,
    target_size=IMG_SIZE,
    subset='training'
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/seg_train/seg_train'

In [ ]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='./tb_model_1', histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
model_1 = tf.keras.Sequential([
    layers.Conv2D(8, 6, padding='same', activation='relu', input_shape=(224,224, 3)),
    layers.Conv2D(8, 6, padding='same', activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(16, 5, padding='same', activation='relu'),
    layers.Conv2D(16, 5, padding='same', activation='relu'),https://colab.research.google.com/github/Enchiii/ImageClassification/blob/main/main.ipynb#
    layers.MaxPool2D(),
    layers.Conv2D(32, 4, padding='same', activation='relu'),
    layers.Conv2D(32, 4, padding='same', activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(10, activation='sigmoid')
])

model_1.compile(
    optimizer=Adam(),
    loss=tf.keras.losses.CategoricalCrossentropy,
    metrics=['accuracy']
)

history_1 = model_1.fit(
    train_data,
    epochs=5,
    steps_per_epoch=len(train_data),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[tbCallBack]
)


AttributeError: module 'keras.api._v2.keras.layers' has no attribute 'MaxPoolD'

# Nowa sekcja